# Euclid Q1 Lenses

* **Phil Marshall, Phil Holloway, Ralf Kaehler, Ferro Shao**
* DP1
* data.lsst.cloud
* r29.1.1
* Fri July 11 2025

## Goals

* Extract _ugrizy_ coadd image cutouts for each Euclid Q1 strong lens candidate in the ECDFS and EDFS DP1 fields
* Visualize them as _gri_ color composites.
* Stretch: deconvolve them using the Rubin SharPy by Kaehler et al (in prep)

In [ ]:
from lsst.daf.butler import Butler
import lsst.afw.display as afw_display
import lsst.geom as geom
from lsst.afw.image import MultibandExposure
import numpy as np
from astropy.visualization import make_lupton_rgb
from astropy.table import Table, Column
import matplotlib.pyplot as plt

afw_display.setDefaultBackend('matplotlib')

## Cutout Image Extraction

First we need to make a list (or better, a table) of targets. Then, for each one, we find out which DP1 coadd patch it lies in. (We'll need to choose which patch, for systems that lie in the patch overlap regions and hence in multiple patches.) Then, we loop over patches and bands, uploading a patch image and extracting all the cutouts we can - which will mean getting the image coordinates for each system

In [ ]:
butler = Butler("dp1", collections="LSSTComCam/DP1")
assert butler is not None

In [ ]:
butler.get_dataset_type('deep_coadd').dimensions.required

### Single Sky Position, Single Band

Let's just try extracting a single 32x32 pixel cutout image in one band.

In [ ]:
ra = 59.626134
dec = -49.06175

band = 'i'

Turn the coordinates into an IAU standard object name, we'll need this later:

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord

# Example RA and Dec coordinates
rahms = ra * u.hourangle  # 12 hours, 30 minutes, 36 seconds
decdms = dec * u.deg    # +12 degrees, 24 minutes, 0 seconds

# Create a SkyCoord object
coordinates = SkyCoord(ra=rahms, dec=decdms, frame='icrs')

# Format the coordinates into an IAU-style string
name = (f'EUCLID J{coordinates.ra.to_string(unit=u.hourangle, sep="", precision=1, pad=True)}'
                  f'{coordinates.dec.to_string(sep="", precision=0, alwayssign=True, pad=True)}') #

print(name)

We need to find the tract and patch that this target is in. This approach was adopted from the CST Tutorial ["03a Image Display and Manipulation"](https://github.com/lsst/tutorial-notebooks/blob/main/DP0.2/03a_Image_Display_and_Manipulation.ipynb).

In [ ]:
radec = geom.SpherePoint(ra, dec, geom.degrees)
cutoutSize = geom.ExtentI(32, 32)

skymap = butler.get("skyMap")
tractInfo = skymap.findTract(radec)
patchInfo = tractInfo.findPatch(radec)

patch = tractInfo.getSequentialPatchIndex(patchInfo)
tract = tractInfo.getId()

dataId = {'tract': tract, 'patch': patch, 'band': band}

When testing, it can be useful to upload the whole patch image and inspect it.

In [ ]:
# deep_coadd = butler.get('deep_coadd', band=band, tract=tract, patch=patch)
# coadd

In [ ]:
# fig = plt.figure(figsize=(6,6))
# display = afw_display.Display(frame=fig)
# display.scale('asinh', 'zscale')
# display.mtv(coadd.image)
# plt.show()

Now to define a small bounding box, and extract the pixels in it. This first cell below _should_ work, but doesn't - maybe some tract/patch confusion. There could be some speed up here at some point, making multiple cutouts from the same patch image using the calexp object's native factory method. See the Cutout Factory demo notebook by Melissa Graham at https://github.com/lsst/cst-dev/blob/main/MLG_sandbox/random/cutout_factory_demo_2025-06-05.ipynb for a working example of making multiple cutouts from the same patch image.

In [ ]:
# xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
# bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)

# cutout = coadd.Factory(coadd, bbox)

Here's some code that does work: define the bounding box, then just grab that part of the image.

In [ ]:
xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)

parameters = {'bbox': bbox}

cutout = butler.get("deep_coadd", parameters=parameters, dataId=dataId)

Quick look to check we got our object!

In [ ]:
fig = plt.figure(figsize=(3, 3))
display = afw_display.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout.image)
plt.show()

### Single Object, Multiple Bands

Loop over all 6 bands and extract the cutout image in each one.

In [ ]:
bands = ["u","g","r","i","z","y"]
cutout = {}

for band in bands:
    dataId = {'tract': tract, 'patch': patch, 'band': band}
    cutout[band] = butler.get("deep_coadd", parameters=parameters, dataId=dataId)

In [ ]:
cutout

In [ ]:
fig = plt.figure(figsize=(3, 3))
display = afw_display.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout["g"].image)
plt.show()

OK - we have 6 cutouts for this target, so can go ahead and make a color composite - see below for this. It took about 5 secs to make them all: we'll need to keep an eye on this, and return to the `factory` approach to try and speed things up a bit.

## _gri_ Composite Image Visualization

Here's a useful function, adapted from the CST Tutorial ["03a Image Display and Manipulation"](https://github.com/lsst/tutorial-notebooks/blob/main/DP0.2/03a_Image_Display_and_Manipulation.ipynb).

In [ ]:
def showRGB(image, bgr="gri", ax=None, fp=None, figsize=(8,8), stretch=100, Q=1, name=None):
    """Display an RGB color composite image with matplotlib.
    
    Parameters
    ----------
    image : `MultibandImage`
        `MultibandImage` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e. keys of the exps dict) indicating what band
        to use for each channel. If `image` only has three filters then this parameter is ignored
        and the filters in the image are used.
    ax : `matplotlib.axes.Axes`
        Axis in a `matplotlib.Figure` to display the image.
        If `axis` is `None` then a new figure is created.
    figsize: tuple
        Size of the `matplotlib.Figure` created.
        If `ax` is not `None` then this parameter is ignored.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    name: str
        The name of the object/field to be displayed.
    """
    # If the image only has 3 bands, reverse the order of the bands to produce the RGB image
    if len(image) == 3:
        bgr = image.filters
    # Extract the primary image component of each Exposure with the .image property, and use .array to get a NumPy array view.
    rgb = make_lupton_rgb(image_r=image[bgr[2]].array,  # numpy array for the r channel
                          image_g=image[bgr[1]].array,  # numpy array for the g channel
                          image_b=image[bgr[0]].array,  # numpy array for the b channel
                          stretch=stretch, Q=Q)  # parameters used to stretch and scale the pixel values
    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(1,1,1)
    
    plt.axis("off")
    ax.imshow(rgb, interpolation='nearest', origin='lower')

    if name is not None:
        plt.text(0, 31, name, color='white', fontsize=12, horizontalalignment='left', verticalalignment='top')
    
    plt.text(0, 2, 'astropy Lupton '+bgr, color='white', fontsize=12, horizontalalignment='left', verticalalignment='top')
    
    plt.tight_layout();

    return

First we need to package our cutouts into a MultibandExposure object, then we pass that to the RGB composite generation function.

In [ ]:
cutouts = [cutout[band] for band in bands]
multibandexposure = MultibandExposure.fromExposures(bands, cutouts)

In [ ]:
showRGB(multibandexposure.image, bgr='gri', figsize=(3,3), stretch=60, Q=10, name=name)

We can use a more recent method for making color composite images, that was used in the First Look release! Here's an alternative RGB function:

In [ ]:
from lsst.pipe.tasks.prettyPictureMaker import PrettyPictureConfig, PrettyPictureTask
from lsst.pipe.tasks.prettyPictureMaker._task import ChannelRGBConfig

def prettyRGB(images, ax=None, fp=None, figsize=(8,8), stretch=250, Q=0.7, name=None):
    """Display an RGB (irg) color composite image with matplotlib using the prettyPictureMaker pipe task.
    
    Parameters
    ----------
    images : dict
        Dictionary of images to display.
    ax : `matplotlib.axes.Axes`
        Axis in a `matplotlib.Figure` to display the image.
        If `axis` is `None` then a new figure is created.
    figsize: tuple
        Size of the `matplotlib.Figure` created.
        If `ax` is not `None` then this parameter is ignored.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    name: str
        The name of the object/field to be displayed.
    """

    prettyPicConfig = PrettyPictureTask.ConfigClass()
    # Magic from Nate Lust:
    prettyPicConfig.localContrastConfig.doLocalContrast = False
    prettyPicConfig.localContrastConfig.sigma = 30
    prettyPicConfig.localContrastConfig.clarity = 0.8
    prettyPicConfig.localContrastConfig.shadows = 0
    prettyPicConfig.localContrastConfig.highlights = -1.5
    prettyPicConfig.localContrastConfig.maxLevel = 2
    prettyPicConfig.imageRemappingConfig.absMax = 11000
    prettyPicConfig.luminanceConfig.max = 100
    prettyPicConfig.luminanceConfig.stretch = stretch    # from kwargs
    prettyPicConfig.luminanceConfig.floor = 0
    prettyPicConfig.luminanceConfig.Q = Q                # from kwargs
    prettyPicConfig.luminanceConfig.highlight = 0.905882
    prettyPicConfig.luminanceConfig.shadow = 0.12
    prettyPicConfig.luminanceConfig.midtone = 0.25
    prettyPicConfig.doPSFDeconcovlve = False   # sic
    prettyPicConfig.exposureBrackets = None
    prettyPicConfig.colorConfig.maxChroma = 80
    prettyPicConfig.colorConfig.saturation = 0.6
    prettyPicConfig.cieWhitePoint = (0.28, 0.28)
    prettyPicConfig.channelConfig = dict(
        g=ChannelRGBConfig(r=0.0, g=0.0, b=1.0),
        r=ChannelRGBConfig(r=0.0, g=1.0, b=0.0),
        i=ChannelRGBConfig(r=1.0, g=0.0, b=0.0),
    )
    prettyPicTask = PrettyPictureTask(config=prettyPicConfig)
    
    bands = "gri"
    coaddG = images['g']
    coaddR = images['r']
    coaddI = images['i']
    
    prettyPicInputs = prettyPicTask.makeInputsFromExposures(i=coaddI, r=coaddR, g=coaddG)
    coaddRgbStruct = prettyPicTask.run(prettyPicInputs)
    coaddRgb = coaddRgbStruct.outputRGB

    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(1,1,1)
    
    plt.axis("off")
    ax.imshow(coaddRgb, interpolation='nearest', origin='lower')

    if name is not None:
        plt.text(0, 31, name, color='white', fontsize=12, horizontalalignment='left', verticalalignment='top')
    
    plt.text(0, 2, 'PrettyPictureTask '+bands, color='white', fontsize=12, horizontalalignment='left', verticalalignment='top')
    
    plt.tight_layout();

    return

In [ ]:
fig = plt.figure(figsize=(8,4))

ax = fig.add_subplot(1,2,1)
showRGB(multibandexposure.image, bgr='gri', ax=ax, stretch=60, Q=10, name=name)

ax = fig.add_subplot(1,2,2)
prettyRGB(cutout, ax=ax, stretch=750, Q=0.7, name=name)

The `prettyPictureMaker` code does a better job at bringing out the color contrast.

Choosing the stretch and Q, in either method, can be a bit fiddly - this is best done when visualizing the whole set of cutouts in a gallery. This is what we will do next.

## Multiple Objects, Multiple Bands, Gallery of Images

What we want is a function that takes in a table of targets, makes cutouts for them all (if they haven't been made already), and then enables visualization of them as a gallery of RGB color composite images (all on the same stretch).

In [ ]:
class StampCollector():
    def __init__(self):
        self.bands = ['u','g','r','i','z','y']
        self.cutoutSize = geom.ExtentI(32, 32)
        self.cutouts = {}
        butler = Butler("dp1", collections="LSSTComCam/DP1")
        assert butler is not None
        butler.get_dataset_type('deep_coadd')
        self.skymap = butler.get("skyMap")
        return
    
    def get_cutouts(self, targets, bands=None):
        """Read in a table of target RA and Dec positions (in degrees) and make cutout images for them all
        
        Parameters
        ----------
        targets : `astropy.Table`
            Table of target coordinates.
        bands : list of strings
            Which bands to make cutouts in.
        """
        self.targets = targets
        # Make IAU names if they don't exist already:
        if 'name' not in self.targets.columns:
            print("Adding IAU names...")
            self.add_iau_names()

        if bands is not None:
            self.bands = bands

        print("Making cutouts:")

        # Loop over targets and bands, extracting cutouts:
        for i in range(len(self.targets)):

            print("  ", self.targets['name'][i],": ",end="")

            radec = geom.SpherePoint(self.targets['ra'][i], self.targets['dec'][i], geom.degrees)           
            tractInfo = self.skymap.findTract(radec)
            patchInfo = tractInfo.findPatch(radec)
            patch = tractInfo.getSequentialPatchIndex(patchInfo)
            tract = tractInfo.getId()
            
            xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
            bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
            parameters = {'bbox': bbox}

            if not self.targets['name'][i] in self.cutouts:
                self.cutouts[self.targets['name'][i]] = {}
            
            for band in self.bands:

                if not band in self.cutouts[self.targets['name'][i]]:
                    dataId = {'tract': tract, 'patch': patch, 'band': band}
                    try:
                        self.cutouts[self.targets['name'][i]][band] = butler.get("deep_coadd", parameters=parameters, dataId=dataId)
                        print(band,end="")
                    except:
                        print(".",end="")
                else:
                    print(".",end="")

            print()
        
        return
        
    def add_iau_names(self):
        names = []
        for i in range(len(self.targets)):
            rahms = self.targets['ra'][i] * u.hourangle
            decdms = self.targets['dec'][i] * u.deg
            coordinates = SkyCoord(ra=rahms, dec=decdms, frame='icrs')
            names.append((f'EUCLID J{coordinates.ra.to_string(unit=u.hourangle, sep="", precision=1, pad=True)}'
                  f'{coordinates.dec.to_string(sep="", precision=0, alwayssign=True, pad=True)}'))
        self.targets.add_column(names, name='name', index=0)
        return
    
    def make_gallery(self, nx=3, style='Pretty', bgr="gri", stretch=750, Q=0.7):
        """Display a gallery of RGB color composite images with matplotlib.
        
        Parameters
        ----------
        nx : integer
            Number of images in a row of the gallery.
        style : str
            Method to use, Lupton (astropy) or Pretty (Rubin)
        bgr : sequence
            A 3-element sequence of filter names (i.e. keys of the exps dict) indicating what band
            to use for each channel. If `image` only has three filters then this parameter is ignored
            and the filters in the image are used.
        stretch: int
            The linear stretch of the image.
        Q: int
            The Asinh softening parameter.
        """
        # Figure out gallery dimensions:
        rem = len(self.targets) % nx
        ny = int((len(self.targets) - rem) / nx) + 1
        print("Making gallery, ",nx," wide by ",ny," deep.")

        # Create an nx by ny grid of subplots
        width = 12
        fig = plt.figure(figsize=(width, ny*(width/nx)))
        plt.axis("off")

        # Loop over targets, stepping through subplots:
        for k in range(len(self.targets)):
            ax = fig.add_subplot(ny,nx,k+1)
            if style == 'Lupton':
                self.show_RGB(self.targets['name'][k], bgr=bgr, ax=ax, stretch=stretch, Q=Q)
            else:
                assert bgr == "gri"
                self.pretty_RGB(self.targets['name'][k], ax=ax, stretch=stretch, Q=Q)
        
        plt.tight_layout()
        
        return

    def show_RGB(self, name, bgr="gri", ax=None, fp=None, figsize=(8,8), stretch=100, Q=5):
        """Display an RGB color composite image with matplotlib.
        
        Parameters
        ----------
        name : str
            Name of the object/field to display.
        bgr : sequence
            A 3-element sequence of filter names (i.e. keys of the exps dict) indicating what band
            to use for each channel. If `image` only has three filters then this parameter is ignored
            and the filters in the image are used.
        ax : `matplotlib.axes.Axes`
            Axis in a `matplotlib.Figure` to display the image.
            If `axis` is `None` then a new figure is created.
        figsize: tuple
            Size of the `matplotlib.Figure` created.
            If `ax` is not `None` then this parameter is ignored.
        stretch: int
            The linear stretch of the image.
        Q: int
            The Asinh softening parameter.
        """
        bands = [bgr[0],bgr[1],bgr[2]]
        cutouts = [self.cutouts[name][band] for band in bands]
        image = MultibandExposure.fromExposures(bands, cutouts).image
        
        # Extract the primary image component of each Exposure with the .image property, and use .array to get a NumPy array view.
        rgb = make_lupton_rgb(image_r=image[bgr[2]].array,  # numpy array for the r channel
                              image_g=image[bgr[1]].array,  # numpy array for the g channel
                              image_b=image[bgr[0]].array,  # numpy array for the b channel
                              stretch=stretch, Q=Q)  # parameters used to stretch and scale the pixel values
        if ax is None:
            fig = plt.figure(figsize=figsize)
            ax = fig.add_subplot(1,1,1)
        
        plt.axis("off")
        ax.imshow(rgb, interpolation='nearest', origin='lower')
    
        if name is not None:
            plt.text(0, 31, name, color='white', fontsize=16, horizontalalignment='left', verticalalignment='top')
        
        plt.text(0, 2, bgr, color='white', fontsize=16, horizontalalignment='left', verticalalignment='top')

        del image, cutouts, rgb
        return

    def pretty_RGB(self, name, ax=None, fp=None, figsize=(8,8), stretch=750, Q=0.7):
        """Display an RGB (irg) color composite image with matplotlib using the prettyPictureMaker pipe task.
        
        Parameters
        ----------
        name : str
            Name of the object/field to display.
        ax : `matplotlib.axes.Axes`
            Axis in a `matplotlib.Figure` to display the image.
            If `axis` is `None` then a new figure is created.
        figsize: tuple
            Size of the `matplotlib.Figure` created.
            If `ax` is not `None` then this parameter is ignored.
        stretch: int
            The linear stretch of the image.
        Q: int
            The Asinh softening parameter.
        name: str
            The name of the object/field to be displayed.
        """
    
        prettyPicConfig = PrettyPictureTask.ConfigClass()
        # Magic from Nate Lust:
        prettyPicConfig.localContrastConfig.doLocalContrast = False
        prettyPicConfig.localContrastConfig.sigma = 30
        prettyPicConfig.localContrastConfig.clarity = 0.8
        prettyPicConfig.localContrastConfig.shadows = 0
        prettyPicConfig.localContrastConfig.highlights = -1.5
        prettyPicConfig.localContrastConfig.maxLevel = 2
        prettyPicConfig.imageRemappingConfig.absMax = 11000
        prettyPicConfig.luminanceConfig.max = 100
        prettyPicConfig.luminanceConfig.stretch = stretch    # from kwargs
        prettyPicConfig.luminanceConfig.floor = 0
        prettyPicConfig.luminanceConfig.Q = Q                # from kwargs
        prettyPicConfig.luminanceConfig.highlight = 0.905882
        prettyPicConfig.luminanceConfig.shadow = 0.12
        prettyPicConfig.luminanceConfig.midtone = 0.25
        prettyPicConfig.doPSFDeconcovlve = False   # sic
        prettyPicConfig.exposureBrackets = None
        prettyPicConfig.colorConfig.maxChroma = 80
        prettyPicConfig.colorConfig.saturation = 0.6
        prettyPicConfig.cieWhitePoint = (0.28, 0.28)
        prettyPicConfig.channelConfig = dict(
            g=ChannelRGBConfig(r=0.0, g=0.0, b=1.0),
            r=ChannelRGBConfig(r=0.0, g=1.0, b=0.0),
            i=ChannelRGBConfig(r=1.0, g=0.0, b=0.0),
        )
        prettyPicTask = PrettyPictureTask(config=prettyPicConfig)
        
        bands = "gri"
        coaddG = self.cutouts[name]['g']
        coaddR = self.cutouts[name]['r']
        coaddI = self.cutouts[name]['i']
        
        prettyPicInputs = prettyPicTask.makeInputsFromExposures(i=coaddI, r=coaddR, g=coaddG)
        coaddRgbStruct = prettyPicTask.run(prettyPicInputs)
        coaddRgb = coaddRgbStruct.outputRGB
    
        if ax is None:
            fig = plt.figure(figsize=figsize)
            ax = fig.add_subplot(1,1,1)
        
        plt.axis("off")
        ax.imshow(coaddRgb, interpolation='nearest', origin='lower')
    
        if name is not None:
            plt.text(0, 31, name, color='white', fontsize=12, horizontalalignment='left', verticalalignment='top')
        
        plt.text(0, 2, bands, color='white', fontsize=12, horizontalalignment='left', verticalalignment='top')
        
        plt.tight_layout();
    
        return

With our `StampCollector` ready, we can prepare and provide a table of targets. Here's a simple example with just two Euclid lens candidates. 

In [ ]:
target_table = Table(names=('ra', 'dec'), dtype=('f4', 'f4'))
target_table.add_row((59.496380, -48.494726))
target_table.add_row((59.626134, -49.06175))

In [ ]:
stamp_collector = StampCollector()
stamp_collector.get_cutouts(target_table,bands=['g','r','i'])

In [ ]:
stamp_collector.targets

In [ ]:
stamp_collector.make_gallery(stretch=750,Q=0.7)

Now let's get all the Euclid lens candidates, in all DP1 fields. These can be downloaded as CSV from a Google sheet on the web, using the pandas native `read_csv` function.

In [ ]:
import pandas as pd

sheet_id = '1OInIecou_c2NqVdpTBSYB2V_j2vanKvKrTBf0qtLCpE' 
csv_url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv' 

# Read the data into a pandas DataFrame
df = pd.read_csv(csv_url)

# Convert the DataFrame to an Astropy Table
all_targets = Table.from_pandas(df)[['Euclid_RA', 'Euclid_Dec']] 
all_targets.rename_column('Euclid_RA', 'ra')
all_targets.rename_column('Euclid_Dec', 'dec')
print(all_targets)

In [ ]:
all_stamp_collector = StampCollector()
all_stamp_collector.get_cutouts(all_targets, bands=['g','r','i'])

In [ ]:
all_stamp_collector.make_gallery(nx=3,style="Pretty",stretch=750,Q=0.7)

## Further Work

* Investigate the Cutout Factory approach - if we can get the world coordinate to image coordinate transformation right, this should give us a bit of a speed up for cases where we have multiple targets in the same patch.
* The gallery needs work: better RGB representation (try and follow the First Look set up?), and a more efficient packing of small images (including scaled fontsize for the overlays).
* Add a method to package ugrizy cutouts into the right format for the Rubin SharPy - and then run that code to sharpen up the images and look for lensed arcs!